In [1]:
import pandas
import transformers

# Add tokens from dataset

In [2]:
tokenizer = transformers.AutoTokenizer.from_pretrained('gpt2-xl')
print(tokenizer.vocab_size)
print(len(tokenizer))


50257
50257


In [3]:
tokenizer.add_special_tokens(
  {'eos_token':'<|startoftext|>',
   'bos_token':'<|startoftext|>'
  }
)

print(tokenizer.vocab_size)
print(len(tokenizer))

50257
50258


In [4]:
tokenizer.add_tokens(['[prompt]','[response]'])

print(tokenizer.vocab_size)
print(len(tokenizer))

50257
50260


In [12]:
tokenizer.save_pretrained('data/modeling/gpt2_xl_model/')

('data/modeling/gpt2_xl_model/vocab.json',
 'data/modeling/gpt2_xl_model/merges.txt',
 'data/modeling/gpt2_xl_model/special_tokens_map.json',
 'data/modeling/gpt2_xl_model/added_tokens.json')

# Add GPT2 model to local

In [11]:
model = transformers.AutoModelWithLMHead.from_pretrained('gpt2-xl')
print(f"Total parameters: {model.num_parameters()/1e6:.2f}M")

Total parameters: 1557.62M


In [9]:
model.resize_token_embeddings(len(tokenizer))

Embedding(50260, 1600)

In [13]:
model.save_pretrained('data/modeling/gpt2_xl_model/')

# Finetune

Due to the method in which Jupyter processes shell commands it won't show STDOUT live, only outputting once the run is finished. So I prefer to just paste this into a terminal instead of running in here.

In [ ]:
!python transformers/examples/language-modeling/run_language_modeling.py \
--output_dir gpt2_distil_output \
--model_type gpt2 \
--model_name_or_path "data/modeling/gpt2_distil_model/" \
--do_train \
--train_data_file "data/scraped/name_desc_nlp_ready_train.txt" \
--do_eval \
--eval_data_file "data/scraped/name_desc_nlp_ready_test.txt" \
--per_gpu_train_batch_size 5

# Scratchpad

### Find unknown tokens in the dataset

In [17]:
import pandas as pd

dataset = pd.read_csv('data/scraped/name_desc_nlp_ready.txt', sep='\t', header=None)
print(dataset.shape)

(125787, 5)


In [28]:
total_tokens = 0
total_unknown_tokens = 0
for ix, row in dataset.iterrows():
  #print("-"*50)
  #print(row[2])
  tokenized_row = tokenizer.encode(row[2])
  #print(tokenized_row)
  total_tokens += len(tokenized_row)
  total_unknown_tokens += tokenized_row.count(50256)

In [36]:
print(total_unknown_tokens / total_tokens)

0.0


### Compare GPT2 Models from HuggingFace

In [54]:
gpt2_distilled = transformers.AutoModelForCausalLM.from_pretrained('distilgpt2')
print(f"Total parameters: {gpt2_distilled.num_parameters()/1e6:.2f}M")

Total parameters: 81.91M


In [53]:
gpt2 = transformers.AutoModelForCausalLM.from_pretrained('gpt2')
print(f"Total parameters: {gpt2.num_parameters()/1e6:.2f}M")

Total parameters: 124.44M


In [49]:
gpt2_medium = transformers.AutoModelForCausalLM.from_pretrained('gpt2-medium')
print(f"Total parameters: {gpt2_medium.num_parameters()/1e6:.2f}M")

Total parameters: 354.82M


In [55]:
gpt2_large = transformers.AutoModelForCausalLM.from_pretrained('gpt2-large')
print(f"Total parameters: {gpt2_large.num_parameters()/1e6:.2f}M")

Total parameters: 774.03M


In [56]:
gpt2_xl = transformers.AutoModelForCausalLM.from_pretrained('gpt2-xl')
print(f"Total parameters: {gpt2_xl.num_parameters()/1e6:.2f}M")

Total parameters: 1557.61M


In [2]:
1500/350

4.285714285714286